In [1]:
from ifm3dpy import O3RCamera, FrameGrabber, ImageBuffer
import json
import matplotlib.pyplot as plt
import cv2
import open3d as o3d
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from glob import glob
from PIL import Image
from time import time
%matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
''' Utilities '''

def timer(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {1000*(t2-t1):.0f}ms')
        return result
    return wrap_func

def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16, 8))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')) + f' {image.shape} {image.shape[1] / image.shape[0]:.2f}'.title())
        plt.imshow(image)
    plt.show()
    
def load_pickle(filename = 'bag'):
    with open(filename, "rb") as fp:
            frames = pickle.load(fp)
    return frames

''' Calibration '''

# @timer
def collect_calibration_points(img):
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)
    
    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        cv2.drawChessboardCorners(img, (9,6), corners2, ret)
        
        return (img, objp, corners)
    return (None, None, None)

# @timer
def calibrate(objpoints, imgpoints, im_size):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, im_size, None, None)
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, im_size, 0, im_size)
    
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        mean_error += error
    print( "total error: {}".format(mean_error/len(objpoints)) )
    return (mtx, newcameramtx, roi, dist)

def undistort(img, config):
    mtx, newcameramtx, roi, dist = config
    dst = cv2.undistort(img, mtx, dist, newcameramtx)
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

In [4]:
o3r = O3RCamera()
config = o3r.get()
config['ports']['port0']['acquisition']['framerate'] = 2
config['ports']['port2']['acquisition']['framerate'] = 2
config['ports']['port0']['state'] = 'RUN'
config['ports']['port2']['state'] = 'RUN'
#config['ports']['port0']['processing']['diParam']['maxDistNoise'] = 0.02
fg3d = FrameGrabber(o3r, 10, 50010)
im3d = ImageBuffer()
fg2d = FrameGrabber(o3r, 10, 50012)
im2d = ImageBuffer()

o3r.set(config)
print(json.dumps(config,indent=4))

{
    "device": {
        "clock": {
            "currentTime": 1581090675006104096
        },
        "diagnostic": {
            "temperatures": [],
            "upTime": 30000000000
        },
        "info": {
            "device": "0301",
            "deviceTreeBinaryBlob": "tegra186-quill-p3310-1000-c03-00-base.dtb",
            "features": {},
            "name": "",
            "partNumber": "M03975",
            "productionState": "AA",
            "serialNumber": "00020142B090",
            "vendor": "0001"
        },
        "network": {
            "authorized_keys": "",
            "ipAddressConfig": 0,
            "macEth0": "48:B0:2D:3A:68:01",
            "macEth1": "00:02:01:42:B0:90",
            "networkSpeed": 1000,
            "staticIPv4Address": "192.168.0.69",
            "staticIPv4Gateway": "192.168.0.201",
            "staticIPv4SubNetMask": "255.255.255.0",
            "useDHCP": false
        },
        "state": {
            "errorMessage": "",
           

In [7]:
objpoints = []
imgpoints = []
w, h = (1280, 800)
while True:
    success2d = fg2d.wait_for_frame(im2d)
    #frame = cv2.imdecode(im2d.jpeg_image(), cv2.IMREAD_UNCHANGED)
    #rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
    
    rgb = cv2.imdecode(im2d.jpeg_image(), cv2.IMREAD_UNCHANGED) 
    calibration_img, objp, corners = collect_calibration_points(rgb)
    if calibration_img is not None:
        cv2.imshow('calibration', calibration_img)
        objpoints.append(objp)
        imgpoints.append(corners)
    k = cv2.waitKey(500)
    if k == 27:
        break
print('Calibrating your pickle with ' + str(len(imgpoints)) + ' images')
config = calibrate(objpoints, imgpoints, (w, h))
with open('rgb_camera_config', 'wb') as f:
    pickle.dump(config, f)

Calibrating your pickle with 68 images
total error: 0.17139463105099997
